In [1]:
import csv
import math
import random
from sklearn.model_selection import train_test_split
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

In [2]:
#read csv
results = []
with open("fifa20.csv", encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile) # change contents to floats
    for row in reader: # each row is a list
        results.append(row)

In [3]:
'''
for i in range(len(results[0])):
    print(str(results[0][i]) + ' - ' + str(results[1][i]))
'''
#read data per column for X and y
X_attr = ['overall','potential','international_reputation','player_tags','player_traits']
#X_attr = ['overall','international_reputation']
y_attr = 'value_eur'
X_attr_index = []
y_attr_index = results[0].index(y_attr)
for i in range(len(results[0])):
    if results[0][i] in X_attr:
        X_attr_index.append(i)

In [4]:
#read data per column for X and y too
X_data = []
y_data = []
for i in range(1,len(results)):
    y_data.append(float(results[i][y_attr_index]))
    row = []
    for j in X_attr_index:
        row.append(results[i][j])
    X_data.append(row)

In [5]:
#cleaning X data, normalizing value so that everything is in the same range which is [0 - 10]
for i in range(len(X_data)):
    '''
    X_data[i][0] = int(X_data[i][0]) / 10
    X_data[i][1] = int(X_data[i][1]) * 2
    '''
    X_data[i][0] = int(X_data[i][0]) / 10
    X_data[i][1] = int(X_data[i][1]) / 10
    X_data[i][2] = int(X_data[i][2]) * 2
    if X_data[i][3]:
        X_data[i][3] = len(str(X_data[i][3]).split(','))
    else:
        X_data[i][3] = 0
    if X_data[i][4]:
        X_data[i][4] = len(str(X_data[i][4]).split(','))
    else:
        X_data[i][4] = 0
    

In [6]:
#combine X and y into 1 array, and also convert the y [in the all data] to scale 0 - 10
max_y = max(y_data)
all_data = []
for i in range(len(X_data)):
    row = []
    row.append((y_data[i] / max_y) * 10)
    for j in range(len(X_data[i])):
        row.append(X_data[i][j])
    all_data.append(row)

In [7]:
#cleaning y data, remove any 0 value data
num = []
for i in range(len(y_data)):
    if y_data[i] == 0:
        num.append(i)
num.sort(reverse=True)
for i in num:
    X_data.pop(i)
    y_data.pop(i)
    all_data.pop(i)

In [8]:
#eucledian distance
def euclidean_distance(x, y):
    return math.sqrt(sum([(a - b) ** 2 for a, b in zip(x, y)]))

In [9]:
#select n cluster and convert the y data into n value based on range, all range have the same size, like 1-10 and 11-20
cluster_num = 3 #number of cluster
min_y = min(y_data)
max_y = max(y_data)
step = max_y/cluster_num
converted_y_data_og = []
for i in range(len(y_data)):
    total = 0
    for z in range(cluster_num):
        total += step
        if y_data[i] <= total and y_data[i] >= (total - step):
            converted_y_data_og.append(z)
            break

In [10]:
#k-means clustering
centroid = []
selected = []
#X_data = sorted(X_data, key = lambda x: random.random())
for i in range(cluster_num):
    index = random.randint(0, len(all_data))
    while(True):
        if index not in selected:
            centroid.append(all_data[index])
            selected.append(index)
            break
print(centroid)
pp = 0
loop_limit = 100
while(True):
    pp += 1
    centroid_member = []
    for z in centroid:
        centroid_member.append([])
    for i in all_data:
        closest = 0
        closest_distance = euclidean_distance(i, centroid[0])
        for z in range(1,len(centroid)):
            distance = euclidean_distance(i, centroid[z])
            if distance < closest_distance:
                closest = z
        centroid_member[closest].append(i)
    #print(centroid_member[0][1])
    not_changed = True
    for i in range(len(centroid_member)):
        if centroid_member[i]:
            total = []
            for z in all_data[0]:
                total.append(0)
            for j in range(len(centroid_member[i])):
                for k in range(len(centroid_member[i][j])):
                    total[k] += centroid_member[i][j][k]
            size = len(centroid_member[i])
            centroid_new = []
            for j in range(len(total)):
                centroid_new.append(total[j]/size)
            if centroid_new != centroid[i]:
                centroid[i] = centroid_new
                not_changed = False
    if pp > loop_limit or not_changed:
        break
print(centroid)

[[0.9004739336492891, 7.6, 8.2, 2, 0, 0], [0.6635071090047394, 7.4, 7.8, 2, 0, 0], [0.31279620853080564, 7.2, 7.2, 2, 0, 2]]
[[0.3710457100487846, 7.088715644820371, 7.422040169133196, 2.004228329809725, 0.15248414376321354, 2.041490486257928], [1.327849151242071, 7.8129827444535636, 7.914050944946572, 4.611339359079704, 0.5521774856203779, 2.3664749383730483], [0.09853345057678131, 6.373224840715296, 7.007829891763333, 2.0391494588163046, 0.04751669609273048, 0.1891456206340677]]


In [11]:
#"testing" the clustering, convert the y data into n value butt the range for each value is based on the min and max of the centroid
centroid_member_min_max = []
for z in centroid:
    centroid_member_min_max.append([])
for i in range(len(all_data)):
    closest = 0
    closest_distance = euclidean_distance(all_data[i], centroid[0])
    for z in range(1,len(centroid)):
        distance = euclidean_distance(all_data[i], centroid[z])
        if distance < closest_distance:
            closest = z
    if centroid_member_min_max[closest]:
        if centroid_member_min_max[closest][0] > y_data[i]:
            centroid_member_min_max[closest][0] = y_data[i]
        if centroid_member_min_max[closest][1] < y_data[i]:
            centroid_member_min_max[closest][1] = y_data[i]
    else:
        centroid_member_min_max[closest].append(y_data[i])
        centroid_member_min_max[closest].append(y_data[i])
    #centroid_member[closest].append(all_data[i])
converted_y_data = []
for i in range(len(y_data)):
    for z in range(len(centroid_member_min_max)):
        if y_data[i] <= centroid_member_min_max[z][1] and y_data[i] >= centroid_member_min_max[z][0]:
            converted_y_data.append(z)
            break
#print(converted_y_data)
guess_r = 0
for i in range(len(converted_y_data)):
    if converted_y_data_og[i] == converted_y_data[i]:
        guess_r += 1
print('accuracy: ' + str(guess_r/len(converted_y_data)))

accuracy: 0.9984468604393166


In [12]:
#split data
X_train, X_test, y_train, y_test = train_test_split(X_data, converted_y_data, test_size=0.2)

In [13]:
# training a KNN classifier 
from sklearn.neighbors import KNeighborsClassifier 
knn = KNeighborsClassifier(n_neighbors = 10).fit(X_train, y_train) 
  
# accuracy on X_test 
accuracy = knn.score(X_test, y_test) 
print(accuracy)

0.9947310038824182


In [14]:
#from this bit below is my attempt at plotting a 3d graph

In [15]:
%matplotlib widget

ModuleNotFoundError: No module named 'ipympl'

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
xs = []
ys = []
zs = []
for i in all_data:
    xs.append(i[0])
    ys.append(i[1])
    zs.append(i[2])
xs2 = []
ys2 = []
zs2 = []
for i in centroid:
    xs2.append(i[0])
    ys2.append(i[1])
    zs2.append(i[2])
ax.scatter(xs2, ys2, zs2, c='red')
#ax.scatter(xs, ys, zs, c='black')
ax.set_xlabel('Overall')
ax.set_ylabel('Int Rep')
ax.set_zlabel('Value')

plt.show()